# Python 扩展模块

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#简介" data-toc-modified-id="简介-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>简介</a></span></li><li><span><a href="#产生一个扩展模块" data-toc-modified-id="产生一个扩展模块-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>产生一个扩展模块</a></span></li><li><span><a href="#手动编译扩展模块" data-toc-modified-id="手动编译扩展模块-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>手动编译扩展模块</a></span></li><li><span><a href="#使用-setup.py-进行编译" data-toc-modified-id="使用-setup.py-进行编译-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>使用 setup.py 进行编译</a></span></li><li><span><a href="#使用编译的模块" data-toc-modified-id="使用编译的模块-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>使用编译的模块</a></span></li><li><span><a href="#压缩生成文件" data-toc-modified-id="压缩生成文件-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>压缩生成文件</a></span></li></ul></div>

## 简介

C Library|	Interface|	Python
:-------------|:-------|:-------
c header| Wrapper C ⇆ Python |import fact 
c implementation	|communication between py + c|fact.fact(10)
	


`Python` 扩展模块将 `PyInt(10)` 转化为 `CInt(10)` 然后调用 C 程序中的`fact() `函数进行计算，再将返回的结果转换回 `PyInt`。

## 产生一个扩展模块

In [1]:
%%file fact.h
#ifndef FACT_H
#define FACT_h
int fact(int n);
#endif

Writing fact.h


In [2]:
%%file fact.c
#include "fact.h"
int fact(int n)
{
    if (n <= 1) return 1;
    else return n * fact(n - 1);
}

Writing fact.c


**包装函数**

In [3]:
%%file fact_wrap.c

/* Must include Python.h before any standard headers*/
#include <Python.h>
#include "fact.h"
static PyObject* wrap_fact(PyObject *self, PyObject *args)
{
    /* Python->C data conversion */
    int n, result;
    // the string i here means there is only one integer
    if (!PyArg_ParseTuple(args, "i", &n))
        return NULL;
    
    /* C Function Call */
    result = fact(n);
    
    /* C->Python data conversion */
    return Py_BuildValue("i", result);
}

/* Method table declaring the names of functions exposed to Python*/
static PyMethodDef ExampleMethods[] = {
    {"fact",  wrap_fact, METH_VARARGS, "Calculate the factorial of n"},
    {NULL, NULL, 0, NULL}        /* Sentinel */
};

/* Module initialization function called at "import example"*/
PyMODINIT_FUNC 
initexample(void)
{
    (void) Py_InitModule("example", ExampleMethods);
}

Writing fact_wrap.c


## 手动编译扩展模块

手动使用 `gcc` 编译，`Windows` 下如果没有 `gcc`，可以通过 `conda `进行安装：
```
conda install mingw4
```
`Window 64-bit` 下编译需要加上 `-DMS_WIN64 `的选项，`include `和 `lib `文件夹的路径对应于本地 `Python `安装的环境：

我的电脑win32才能编译，还有一个问题，直接在jupyter里面识别不了gcc,conda命令
```
gcc -DMS_WIN32 -c fact.c fact_wrap.c -ID:\Python\include
gcc -DMS_WIN32 -shared fact.o fact_wrap.o -LD:\Python\libs -lpython27 -o example.pyd
```

In [4]:
!ls

01-introduction.ipynb
02-python-extension-modules.ipynb
03-cython.ipynb
03-cython-sum.zip
03-particle.zip
04-profiling-with-annotations.ipynb
05-ctypes.ipynb
example.zip
extern.zip
fact.c
fact.h
fact_wrap.c
fib.zip
fib_orig.c
fib_orig.html
fib_orig.pyx


In [5]:
# 导入这个模块
import example
print dir(example)

['__doc__', '__file__', '__name__', '__package__', 'fact']


In [6]:
# 使用里面的函数
example.fact(10)

3628800

In [7]:
#清理刚才生成的文件：
!rm -f example.pyd

## 使用 setup.py 进行编译

In [8]:
%%file setup.py
from distutils.core import setup, Extension
ext = Extension(name='example', sources=['fact_wrap.c', 'fact.c'])
setup(name='example', ext_modules=[ext])

Writing setup.py


使用 `distutils` 中的函数，我们进行 `build `和` install`：
```
python setup.py build (--compiler=mingw64)
python setup.py install
```
也可以直接：`!python setup.py build_ext --inplace`

In [9]:
!python setup.py build_ext --inplace

running build_ext
building 'example' extension
creating build
creating build\temp.win32-2.7
creating build\temp.win32-2.7\Release
D:\MinGW\bin\gcc.exe -mdll -O -Wall -ID:\Python\include -ID:\Python\PC -c fact_wrap.c -o build\temp.win32-2.7\Release\fact_wrap.o
D:\MinGW\bin\gcc.exe -mdll -O -Wall -ID:\Python\include -ID:\Python\PC -c fact.c -o build\temp.win32-2.7\Release\fact.o
writing build\temp.win32-2.7\Release\example.def
D:\MinGW\bin\gcc.exe -shared -s build\temp.win32-2.7\Release\fact_wrap.o build\temp.win32-2.7\Release\fact.o build\temp.win32-2.7\Release\example.def -LD:\Python\libs -LD:\Python\PCbuild -LD:\Python\PC\VS9.0 -lpython27 -lmsvcr90 -o E:\Project_Sources\notebook\other-languages\example.pyd


## 使用编译的模块

In [10]:
import example
print 'factorial of 10:', example.fact(10)

factorial of 10: 3628800


In [11]:
def pyfact(n):
    if n <= 1: return 1
    return n * pyfact(n-1)

print pyfact(10)
print example.fact(10)

3628800
3628800


**时间测试：**

In [12]:
%timeit example.fact(10)

The slowest run took 8.16 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 229 ns per loop


In [13]:
%timeit pyfact(10)

1000000 loops, best of 3: 1.74 µs per loop


In [14]:
example.fact(100)

0

In [15]:
# 会出现溢出的结果，因为 int 表示的值有限，但是 pyfact 不会有这样的问题：
pyfact(100)

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000L

## 压缩生成文件

In [16]:
import zipfile
names = 'fact.c fact.h fact_wrap.c example.pyd setup.py fib_orig.c fib_orig.html fib_orig.pyx'.split()
with zipfile.ZipFile('example.zip','w',zipfile.ZIP_DEFLATED)as f:
    for name in names:
        f.write(name)

In [17]:
import os
for name in names:
    os.remove(name)

In [18]:
!rm -rf build
# shutil.rmtree("build")

文件操作见笔记：[shutil 模块](../python-tools/08-shutil.ipynb)